# <center> InternAnalytics </center> #

### 1. Introduction ###
Objectif :
Le projet InternAnalytics a pour but d’analyser une base de données regroupant des offres d'emploi afin de tirer des insights exploitables pour :

- Comprendre les tendances du marché de l’emploi (secteurs les plus actifs, localisations recherchées, etc.).
- Identifier les facteurs d’attractivité des offres d’emploi (entreprises populaires, offres les plus vues).
- Évaluer les tendances salariales en fonction des secteurs, des localisations, et des postes proposés.
- Offrir des recommandations exploitables, par exemple :
    -  Aux chercheurs d’emploi : quels secteurs et localisations privilégier.
    - Aux recruteurs : comment optimiser leurs offres pour attirer plus de candidats.
- S’informer sur les offres d’alternance :
    - Identifier les entreprises qui proposent des contrats en alternance.
    - Analyser les postes et secteurs qui recrutent le plus en alternance.
    - Comprendre les salaires et avantages liés aux offres d’alternance.
- Champs disponibles : `job_id`, `company_name`, `title`, `description`, `max_salary`, `pay_period`, `location`, `company_id`, `views, med_salary`.

Ce projet vise donc à offrir une vision globale et spécifique des opportunités sur le marché, avec un focus particulier sur les alternances, pour mieux répondre aux besoins des étudiants et jeunes professionnels.


### 2. Chargement des données ###
- Importation des bibliothèques nécessaires (e.g., pandas, numpy, matplotlib, seaborn).
- Chargement du fichier de données (CSV, SQL, ou autre).
- Aperçu des données (df.head() et df.info()).

In [46]:
# Utility import
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Dataset import
import os
import json
from kaggle.api.kaggle_api_extended import KaggleApi

In [47]:
# Dowload dataset using kaggle API
path = os.getcwd().replace("notebooks", "kaggle.json")

with open(path, 'r') as file:
    kaggle_creds = json.load(file)

os.environ["KAGGLE_USERNAME"] = kaggle_creds["username"]
os.environ["KAGGLE_KEY"] = kaggle_creds["key"]

dataset_name = 'arshkon/linkedin-job-postings'
download_path = "../data/raw"
os.makedirs(download_path, exist_ok=True)

api = KaggleApi()
api.authenticate()
print("Téléchargement en cours...")
api.dataset_download_files(dataset_name, path=download_path, unzip=True)
print(f"Dataset téléchargé et extrait dans : {download_path}")

Téléchargement en cours...
Dataset URL: https://www.kaggle.com/datasets/arshkon/linkedin-job-postings
Dataset téléchargé et extrait dans : ../data/raw


### 3. Nettoyage et Préparation des Données ###
- Détection des données manquantes : Vérifier les colonnes avec des valeurs NaN ou vides.
- Traitement des doublons : Vérifier et supprimer les entrées redondantes si nécessaire.
- Standardisation des colonnes : Par exemple, uniformiser les formats des - salaires (max_salary, med_salary) ou des localisations (location).
- Conversion des types de données pour faciliter l’analyse :
    - pay_period en fréquence (mensuel, annuel, etc.).
views en entier.
    - Extraction de catégories pertinentes à partir de description (ex : analyse des mots-clés).

In [49]:
# Charger les donnée dans un dataframe pandas 
data = pd.read_csv("../data/raw/postings.csv")
data.head()

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,38480.0,8540.0,34021.0
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,NaN,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,83200.0,80521.0,8069.0
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,...,We are currently accepting resumes for FOH - A...,1.713278e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,55000.0,45202.0,39061.0
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,...,This position requires a baseline understandin...,1.712896e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,157500.0,11040.0,36059.0
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,...,NaN,1.713452e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,70000.0,52601.0,19057.0


In [50]:
data_clean = data.drop_duplicates()
data_clean = data_clean.drop(columns=['med_salary', 'min_salary', 'max_salary', 'listed_time', 'posting_domain', 'original_listed_time', 'job_posting_url', 'expiry', 'closed_time', 'application_url' ,'sponsored', 'zip_code', 'fips', 'compensation_type'])
data_clean.head()

,job_id,company_name,title,description,pay_period,location,company_id,views,formatted_work_type,applies,remote_allowed,application_type,formatted_experience_level,skills_desc,work_type,currency,normalized_salary
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,HOURLY,"Princeton, NJ",2774458.0,20.0,Full-time,2.0,NaN,ComplexOnsiteApply,NaN,Requirements: \n\nWe are seeking a College or ...,FULL_TIME,USD,38480.0
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",HOURLY,"Fort Collins, CO",NaN,1.0,Full-time,NaN,NaN,ComplexOnsiteApply,NaN,NaN,FULL_TIME,USD,83200.0
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,YEARLY,"Cincinnati, OH",64896719.0,8.0,Full-time,NaN,NaN,ComplexOnsiteApply,NaN,We are currently accepting resumes for FOH - A...,FULL_TIME,USD,55000.0
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,YEARLY,"New Hyde Park, NY",766262.0,16.0,Full-time,NaN,NaN,ComplexOnsiteApply,NaN,This position requires a baseline understandin...,FULL_TIME,USD,157500.0
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,YEARLY,"Burlington, IA",NaN,3.0,Full-time,NaN,NaN,ComplexOnsiteApply,NaN,NaN,FULL_TIME,USD,70000.0


In [51]:
data_clean['remote_allowed'] = data_clean['remote_allowed'].fillna(0).astype(int)
data_clean['applies'] = data_clean['applies'].fillna(0).astype(int)
for elt in data_clean :
    data_clean[elt] = data_clean[elt].fillna('Not specified')

data_clean = data_clean.dropna(subset=['company_name'])
download_path = "../data/interim/posting_clean.csv"

os.makedirs(download_path, exist_ok=True)
data_clean.to_csv(download_path, header=True)

### 4. Exploration des Données (EDA - Exploratory Data Analysis) ###
- Statistiques descriptives :
    - Distribution des salaires (max_salary, med_salary).
    - Entreprises avec le plus d’offres (company_name).
- Visualisation des données :
    - Histogrammes et boxplots pour les salaires.
    - Nuage de points pour corréler les salaires et les vues.
    - Graphiques géographiques pour la localisation des offres (location).
- Analyse des tendances :
    - Tendances par secteur (basé sur les mots-clés dans title ou description).
    - Entreprises les plus attractives (basé sur views).

### 5. Analyses Avancées et Insights ###

- Segmentation des données :
    - Comparaison des salaires par localisation.
    - Entreprises par secteur d’activité.
- Détection des relations et corrélations :
    - Corrélation entre views et max_salary.
    - Analyse des types d’emploi les plus attractifs (title, description).
- Suggestions basées sur les insights :
    - Quels types d’emplois attirer le plus de vues ?
    - Quels secteurs offrent les meilleurs salaires ?